<a href="https://colab.research.google.com/github/AmnNrz/GEE_tutorial/blob/main/Time_Series_Change_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the geemap package with additional features used in workshops
%pip install -U "geemap[workshop]"

In [ ]:
# Import required libraries
import ee         # Earth Engine API
import geemap     # Helper library to work with Earth Engine in Python

# Update geemap in case it's not the latest version
geemap.update_package()

# Authenticate your Earth Engine account (one-time popup for login)
ee.Authenticate()

# Initialize the Earth Engine session (replace with your project if needed)
ee.Initialize(project='clear-shadow-332006')

In [ ]:
# Create an interactive map
map = geemap.Map()
map  # This displays the map in the notebook


In [ ]:
# Get the features (shapes) you drew on the map (e.g., polygon and points)
all_features = map.user_rois
all_features  # Show the list of drawn features

# Convert all features to a list so we can access them individually
features_list = all_features.toList(all_features.size())

# Assign the first feature (likely a polygon) as the study area
study_area = ee.Feature(features_list.get(0)).geometry()

# Assign the second and third features as points (for time series extraction)
point1 = ee.Feature(features_list.get(1)).geometry()
point2 = ee.Feature(features_list.get(2)).geometry()

In [ ]:
# ---------------------------------------------------------------
# Step 1: Load and scale Landsat 8 surface reflectance imagery
# ---------------------------------------------------------------

# Filter Landsat 8 Tier 1 imagery by the study area
l8_sr = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
         .filterBounds(study_area))

# Function to scale optical and thermal bands based on metadata
def apply_scale_factors(image):
    # Scale optical bands (B1–B7)
    optical = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    # Scale thermal band (B10)
    thermal = image.select('ST_B10').multiply(0.00341802).add(149.0)
    # Return image with scaled bands
    return (image
            .addBands(optical, overwrite=True)
            .addBands(thermal,  overwrite=True))

# Apply the scaling function to each image in the collection
l8_sr = l8_sr.map(apply_scale_factors)

In [ ]:
# ---------------------------------------------------------------
# Step 2: Cloud and snow masking using QA_PIXEL band
# ---------------------------------------------------------------

# Function to mask out clouds, cloud shadows, and snow
def mask_l8_sr(image):
    qa = image.select('QA_PIXEL')  # Select quality assurance band
    mask = (qa.bitwiseAnd(1 << 4).eq(0)     # Bit 4 = cloud shadow
            .And(qa.bitwiseAnd(1 << 5).eq(0))  # Bit 5 = snow
            .And(qa.bitwiseAnd(1 << 6).neq(0))) # Bit 6 = clear condition
    return (image.updateMask(mask)  # Apply the mask to the image
                 .select('SR_B.*')  # Keep only surface reflectance bands
                 .copyProperties(image, ['system:time_start']))  # Preserve timestamp


In [ ]:
# ---------------------------------------------------------------
# Step 3: Add EVI (Enhanced Vegetation Index) band
# ---------------------------------------------------------------

# Function to calculate and add EVI band
def add_evi(image):
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('SR_B5'),    # Near Infrared band
            'RED': image.select('SR_B4'),    # Red band
            'BLUE': image.select('SR_B2')    # Blue band
        }).rename('EVI')  # Rename result as 'EVI'
    return image.addBands(evi)  # Add the EVI band to the image

# ---------------------------------------------------------------
# Step 4: Create yearly EVI composite images
# ---------------------------------------------------------------

# Function to create a median composite image for each year
def make_yearly_composite(year):
    year   = ee.Number(year)  # Convert to Earth Engine number
    start  = ee.Date.fromYMD(year, 5, 1)   # Start date: May 1
    end    = ee.Date.fromYMD(year, 9, 15)  # End date: Sep 15

    composite = (l8_sr
                 .filterDate(start, end)  # Filter to summer months
                 .map(mask_l8_sr)         # Apply cloud mask
                 .median())               # Take pixel-wise median

    composite = (add_evi(composite)         # Add EVI band
                 .select('EVI')             # Keep only EVI
                 .set('system:time_start', start.millis()))  # Set image date
    return composite



In [ ]:
# ---------------------------------------------------------------
# Step 5: Build ImageCollection for all years
# ---------------------------------------------------------------

# Create a list of years to process
years = ee.List.sequence(2014, 2020)

# Map the composite function over all years
composites = years.map(make_yearly_composite)

# Convert the list of images into an ImageCollection
yearly_col = ee.ImageCollection.fromImages(composites)

# ---------------------------------------------------------------
# Step 6: Visualize the yearly EVI composites
# ---------------------------------------------------------------

# Define visualization parameters for EVI
evi_vis = {'min': -1, 'max': 1, 'palette': ['white', 'green']}

# Center the map on the study area
map.centerObject(study_area, 8)

# Loop through each year to add its EVI layer to the map
for yr in years.getInfo():   # Convert list to client-side for loop
    img = (yearly_col
           .filterDate(f'{yr}-01-01', f'{yr}-12-31')  # Get image from that year
           .first()  # Take the first image (only one per year)
           .clip(study_area))  # Crop to study area
    map.addLayer(img, evi_vis, f'{yr} EVI')  # Add to map with label

map  # Display map with layers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# STEP 2: Create list of years again (Python list this time)
years = list(range(2014, 2021))

# STEP 3: Define function to extract time series from a point
def extract_time_series(image_collection, point, band_name, scale=30):
    def reduce_image(img):
        value = img.reduceRegion(
            reducer=ee.Reducer.mean(),  # Average value at the point
            geometry=point,
            scale=scale,
            maxPixels=1e13
        )
        return ee.Feature(None, {
            'date': img.date().millis(),     # Store date in milliseconds
            'value': value.get(band_name)    # Extract band value
        })

    return ee.FeatureCollection(image_collection.map(reduce_image))

# STEP 4: Extract time series for both points
ts_point1 = extract_time_series(yearly_col.select('EVI'), point1, 'EVI')
ts_point2 = extract_time_series(yearly_col.select('EVI'), point2, 'EVI')

# STEP 5: Convert to pandas DataFrame
df1 = geemap.ee_to_df(ts_point1)
df1['date'] = pd.to_datetime(df1['date'], unit='ms')  # Convert timestamp

df2 = geemap.ee_to_df(ts_point2)
df2['date'] = pd.to_datetime(df2['date'], unit='ms')


In [ ]:
# STEP 6: Plot EVI time series for both points
plt.figure(figsize=(10, 5))
plt.plot(df1['date'], df1['value'], marker='o', label='Point 1')
plt.plot(df2['date'], df2['value'], marker='x', label='Point 2')
plt.title('EVI Time Series')  # Title of the chart
plt.xlabel('Date')
plt.ylabel('EVI')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------
# Create an animated GIF of EVI change over the years
# ---------------------------------------------------------------

import os
import geemap
from PIL import Image, ImageDraw, ImageFont, ImageSequence

# Step 1: Define animation parameters
video_args = {
    'dimensions': 768,        # Size of the video frame (pixels)
    'region': study_area,     # Area to animate
    'framesPerSecond': 1,     # 1 frame per second
    'bands': ['EVI'],         # Use only EVI band
    'min': -1,
    'max': 1,
}

# Step 2: Download the raw GIF
out_gif = '/content/evi_timeseries.gif'
geemap.download_ee_video(yearly_col.select('EVI'), video_args, out_gif)

# Step 3: Open the GIF and prepare to add year labels
orig = Image.open(out_gif)
years = list(range(2014, 2021))  # Year labels to add
frames = []
font = ImageFont.load_default()  # Use default font

# Loop through each frame to draw year labels
for idx, frame in enumerate(ImageSequence.Iterator(orig)):
    img = frame.convert('RGBA')  # Convert frame to RGBA format
    draw = ImageDraw.Draw(img)
    text = str(years[idx])  # Label for this year
    bbox = draw.textbbox((0, 0), text, font=font)  # Get text size
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    x, y = 10, 10  # Top-left corner
    draw.rectangle([x-4, y-4, x+w+4, y+h+4], fill=(0, 0, 0, 127))  # Background box
    draw.text((x, y), text, font=font, fill=(255, 255, 255, 255))  # Write text
    frames.append(img)  # Save the modified frame

# Step 4: Save new GIF with labels
labeled_gif = '/content/evi_timeseries_labeled.gif'
frames[0].save(
    labeled_gif,
    save_all=True,
    append_images=frames[1:],
    duration=orig.info['duration'],
    loop=0
)

# Step 5: Display the final labeled GIF
geemap.show_image(labeled_gif)